In [ ]:
%install_ext https://raw.github.com/cjdrake/ipython-magic/master/gvmagic.py

In [ ]:
%load_ext gvmagic

In [ ]:
from pyeda.inter import *

In [ ]:
a, b, c = map(bddvar, 'abc')

In [ ]:
f = a & b | a & c | b & c

In [ ]:
f.to_dot()

In [ ]:
%dotobj f

In [ ]:
~a & ~b | ~a & b | a & ~b | a & b

In [ ]:
(~a | ~b) & (~a | b) & (a | ~b) & (a | b)

In [ ]:
f = a ^ b ^ c

In [ ]:
%dotobj f

In [ ]:
f.satisfy_one()

In [ ]:
list(f.satisfy_all())

In [ ]:
X = bddvars('x', 8)

In [ ]:
Y = bddvars('y', 8)

In [ ]:
f1 = X[0] & X[1] | X[2] & X[3] | X[4] & X[5]

In [ ]:
f2 = X[0] & X[3] | X[1] & X[4] | X[2] & X[5]

In [ ]:
%dotobj f1

In [ ]:
%dotobj f2

In [ ]:
g2 = f2.compose({X[0]: Y[0], X[1]: Y[2], X[2]: Y[4], X[3]: Y[1], X[4]: Y[3], X[5]: Y[5]})

In [ ]:
%dotobj g2